In [9]:
import pandas as pd
import numpy as np
from sklearn.cluster import OPTICS

import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
address_url = "C:/Users/abhil/Desktop/Python/DataSet/hi.csv"
address = pd.read_csv(address_url)

In [11]:
df = address[:1000]

In [12]:
df = df.drop(['NUMBER','STREET', 'UNIT','CITY', 'DISTRICT', 'REGION', 'POSTCODE', 'ID', 'HASH'], axis=1)

In [13]:
df = df[['LAT', 'LON']]
coords = df.as_matrix(columns=['LAT', 'LON'])

c:\users\abhil\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [14]:
model = OPTICS(min_samples=9, rejection_ratio=0.5)
db = model.fit_predict(np.radians(coords))
cluster_labels = model.labels_
num_clusters = len(set(cluster_labels))
clusters = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)])
print('Number of clusters: {}'.format(num_clusters))
df = df[['LAT', 'LON']]

Number of clusters: 35


In [15]:
df['Cluster'] = db
cluster_count = df.groupby('Cluster').count().sort_values(by=['LAT'],ascending=False)[:10]
cluster_count = cluster_count.drop(['LON'],axis=1)
x = cluster_count.LAT.keys()


In [17]:
cluster_count

,LAT
Cluster,
-1,270
1,76
14,50
17,48
21,37
31,33
0,32
28,28
13,28


In [18]:
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)


centermost_points = clusters.map(get_centermost_point)
#clusters
#get_centermost_point(clusters[10])
# cluster_centerpoints = [0][0]
# count = 0 
# for i in clusters[:len(clusters)-1]:
#     print(get_centermost_point(i))


NameError: name 'MultiPoint' is not defined

In [ ]:
lats, lons = zip(*centermost_points)
rep_points = pd.DataFrame({'lon':lons, 'lat':lats})

In [ ]:
rs = rep_points.apply(lambda row: df[(df['LAT']==row['lat']) & (df['LON']==row['lon'])].iloc[0], axis=1)
j = 0
rs['count'] = cluster_count.LAT
# for i in cluster_count.LAT:
#     print(i)
#     j+=1
rs.dropna(axis=0)
